In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ.get('SENDGRID_ALT_KEY')

In [2]:
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))


In [ ]:
# test SendGrid mailing list operations
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
params = {'page': 1, 'page_size': 1}
response = sg.client.contactdb.recipients.get(query_params=params)
print(response.status_code)
#print(response.body)
#print(response.headers)


In [ ]:
# Sendgrid retrieve recipients
import json
params = {'page':1, 'page_size':1}
response = sg.client.contactdb.recipients.get(query_params=params)
print(response.status_code)
contacts = json.loads(response.body)
emails = [recipient['email'] for recipient in contacts['recipients']]
emails

In [ ]:
response = sg.client.contactdb.recipients.count.get()
print(response.status_code)
print(response.body)

parse index.xml to get latest feed item

In [8]:
import feedparser

feed = feedparser.parse('../docs/index.xml')
newsletter = feed.entries[0]
print(newsletter.title)
print(newsletter.description)

ADCN: Demand and Capacity, College Sports, Youth Burnout … and More News
<p>Work on developing this newsletter continues. I’m pushing in the direction of something quicker and newsier for 2026. Thanks for reading!</p>
<section class="level3" id="demand-and-capacity">
<h3 class="anchored">Demand and Capacity</h3>
<p>Structural engineers use Demand-Capacity models to understand risk related to buildings and bridges. Capacity refers to the unit’s strength based on its design and material construction. The external forces that the unit experiences is the Demand. When Capacity is greater than Demand we are safe. Athletes’ Injury Risk models based on Training Load data fail to parse Capacity and Demand, neglecting some really useful descriptive details in the process.</p>
<p>Let’s look first at Capacity. Capacity closely approximates Training Load as an indicator of athletes’ fitness. A more complete story for Capacity should also include Readiness parameters that come from recovery, stress 

In [ ]:
import re
from bs4 import BeautifulSoup
soup = BeautifulSoup(newsletter.description, 'html.parser')
sections = soup.find_all('section')
subject = '"subject": "' + newsletter.title + '"'
introduction = '"introduction":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in reversed(sections[0].find_previous_siblings('p'))]) + ']'
#section = '"sections":' + '[' + ",".join([ '{"section":"' + re.sub(r'"', r'\"', str(s)) + '"}' for s in sections]) + ']'
section = []
for s in sections:
    heading = '"heading": "' + re.sub(r'"', r'\"', str(s.find('h3'))) + '"'
    description = '"description": "' + re.sub(r'"', r'\"', str(s.find('p'))) + '"'
    paragraphs = '"paragraphs":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in s.find('p').find_next_siblings('p')]) + ']'
    sec = '{"section": {' + heading + ', ' + description + ', ' + paragraphs + '}}'
    section.append(sec)
h_sections = '"sections": [' + ",".join(section) + ']'
handlebar = '{' + subject + ', ' + introduction + ', ' + h_sections + '}'
print(handlebar)


AttributeError: 'NoneType' object has no attribute 'find_next_siblings'

In [ ]:
import sendgrid
import os
from sendgrid.helpers.mail import *
from dotenv import load_dotenv
load_dotenv()
emails = [] #..fill as needed
for email in emails:
	sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
	message = Mail(from_email=os.environ.get('SENDGRID_SENDER_EMAIL'), to_emails=email)
	message.dynamic_template_data = eval(handlebar)
	message.template_id = os.environ.get('SENDGRID_TEMPLATE_ID')
	response = sg.send(message)
	print(response.status_code, email)